# Advent of Code

## Day 1

Find the top three Elves carrying the most Calories. How many Calories are those Elves carrying in total?

In [ ]:
from itertools import groupby

topk = 3

with open("data/day1.txt", "r") as f:
    data = [l.strip() for l in f.readlines()]
    
sum(sorted(sum(map(int, v)) for k, v in groupby(data, key=lambda l: l != '') if k)[-topk:])

## Day 2

What would your total score be if everything goes exactly according to your strategy guide?

In [ ]:
with open("data/day2.txt", "r") as f:
    data = [l.strip().split(' ') for l in f.readlines()]

first_col = {k: v for v, k in enumerate('ABC')}
second_col = {k: v for v, k in enumerate('XYZ')}
mapping = list(map(lambda x: (first_col[x[0]], second_col[x[1]]), data))

shape_points = map(lambda x: 1 + x[1], mapping)
outcome_points = map(lambda y : 3 * (y + 1), [v if abs(v) != 2 else -v // 2 for v in map(lambda x: x[1] - x[0], mapping)])
sum(shape_points) + sum(outcome_points)

In [ ]:
shape_points = map(lambda x: 1 + ((x[0] + x[1] - 1) % 3), mapping)
outcome_points = map(lambda x: 3 * second_col[x[1]], data)
sum(shape_points) + sum(outcome_points)

## Day 3

Find the item type that appears in both compartments of each rucksack. What is the sum of the priorities of those item types?

In [ ]:
import string

with open("data/day3.txt", "r") as f:
    data = [l.strip() for l in f.readlines()]

mapping = {k: v for v, k in enumerate(string.ascii_letters, 1)}

def get_priorities(string):
    split = len(string) // 2
    a, b = string[:split], string[split:]
    return sum(mapping[item] for item in set(a).intersection(b))

priorities = map(get_priorities, data)
sum(priorities)

In [ ]:
def get_priorities(triplet):
    a, b, c = triplet
    return sum(mapping[item] for item in set(a).intersection(b).intersection(c))

data_groups = zip(*(iter(data),) * 3)

priorities = map(get_priorities, data_groups)
sum(priorities)

## Day 4

In how many assignment pairs does one range fully contain the other?

In [ ]:
with open("data/day4.txt", "r") as f:
    data = [list(map(lambda x: x.split('-'), l.strip().split(','))) for l in f.readlines()]

def fully_overlap(a, b):
    return (int(a[0]) < int(b[0])) ^ (int(a[1]) < int(b[1])) or a[0] == b[0] or a[1] == b[1]
    
sum(fully_overlap(a, b) for a, b in data)

In [ ]:
def overlap(a, b):
    return (int(a[0]) <= int(b[1])) and (int(a[1]) >= int(b[0]))
    
sum(overlap(a, b) for a, b in data)

## Day 5

After the rearrangement procedure completes, what crate ends up on top of each stack?

In [ ]:
with open("data/day5.txt", "r") as f:
    stacks = list(reversed([next(f).strip('\n') for _ in range(8)]))
    stacks_cpy = {k+1: [line[4*k+1] for line in stacks if line[4*k+1] != ' '] for k in range(9)}
    for _ in range(2):
        next(f)
    instructions = [dict(zip(*(iter(l.strip().split()),) * 2)) for l in f.readlines()]

for instr in instructions:
    for _ in range(int(instr['move'])):
        stacks_cpy[int(instr['to'])].append(stacks_cpy[int(instr['from'])].pop())
        
''.join(v[-1] for v in stacks_cpy.values())

In [ ]:
stacks_cpy = {k+1: [line[4*k+1] for line in stacks if line[4*k+1] != ' '] for k in range(9)}

for instr in instructions:
    pack = list(reversed([stacks_cpy[int(instr['from'])].pop() for _ in range(int(instr['move']))]))
    stacks_cpy[int(instr['to'])].extend(pack)
    
''.join(v[-1] for v in stacks_cpy.values())

## Day 6

How many characters need to be processed before the first start-of-packet marker is detected?

In [ ]:
with open("data/day6.txt", "r") as f:
    data = next(f).strip()

[len(set(data[i-4:i])) for i in range(len(data))].index(4)

In [ ]:
[len(set(data[i-14:i])) for i in range(len(data))].index(14)

## Day 7

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [ ]:
from collections import defaultdict

with open("data/day7.txt", "r") as f:
    data = [line.strip().split() for line in f]
    
system = defaultdict(int)

for line in data:
    if line[0] == '$':
        if line[1] == 'cd':
            if line[2] == '/':
                current = ['.']
            elif line[2] == '..':
                current = current[:-1]
            else:
                current.append(line[2])
    if line[0].isdigit():
        for i, _ in enumerate(current, 1):
            system['/'.join(current[:i])] += int(line[0])
            
sum(v for v in system.values() if v < 100000)

In [ ]:
needed = system['.'] - 40000000
min(v for v in system.values() if v >= needed)

## Day 8

Consider your map; how many trees are visible from outside the grid?

In [ ]:
import numpy as np

data = np.genfromtxt("data/day8.txt", delimiter=1)

up = np.diff(np.maximum.accumulate(data, axis=0), prepend=-1, axis=0)
down = np.flip(np.diff(np.maximum.accumulate(np.flip(data, axis=0), axis=0), prepend=-1, axis=0), axis=0)
left = np.diff(np.maximum.accumulate(data, axis=1), prepend=-1, axis=1)
right = np.flip(np.diff(np.maximum.accumulate(np.flip(data, axis=1), axis=1), prepend=-1, axis=1), axis=1)

np.sum(np.logical_or.reduce([up, down, left, right]))

In [ ]:
def vision_(x):
    ahead = np.minimum.accumulate(x[1:] < x[0], axis=0)
    return ahead.sum(axis=0) + (~ahead).any(axis=0)

vision = lambda x: np.array([vision_(x[i:]) for i in range(0, x.shape[0])])

up = vision(data)
down = np.flip(vision(np.flip(data, axis=0)), axis=0)
left = vision(data.T).T
right = np.flip(vision(np.flip(data, axis=1).T).T, axis=1)

np.max(up * down * left * right)

## Day 9

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

In [ ]:
import numpy as np

with open("data/day9.txt", "r") as f:
    data = [line.strip().split(' ') for line in f]
    
k = 2 # Number of knots (set to 10 for part 2)

current = np.zeros((k, 2))
visited = []

move = {'R': np.array([0, 1]), 'L': np.array([0, -1]), 'U': np.array([1, 0]), 'D': np.array([-1, 0])}

for line in data:
    for _ in range(int(line[1])):
        current[0] += move[line[0]]
        head = current[0]
        for i in range(1, len(current)):
            d = head - current[i]
            current[i] += (np.sum(d**2) > 3) * np.sign(d)
            head = current[i]
        visited.append(tuple(head))

len(set(visited))

## Day 10

Render the image given by your program. What eight capital letters appear on your CRT?

In [ ]:
with open("data/day10.txt", "r") as f:
    data = [line.strip() for line in f]
    
X = [1]
for line in data[:-1]:
    if line ==  'noop':
        X.append(X[-1])
    else:
        _, val = line.split()
        X.append(X[-1])
        X.append(X[-1] + int(val))

sum(i * X[i-1] for i in range(20, 260, 40))

In [ ]:
pprint(list(map(lambda x: ''.join(x),
    zip(*(iter('#' if 0 <= 1 + val - (i % 40) < 3 else '.' for i, val in enumerate(X)),) * 40)
)))

## Day 11

Figure out which monkeys to chase by counting how many items they inspect over 20 rounds. What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans?

In [25]:
import re
from math import prod

with open("data/day11.txt", "r") as f:
    data = [line.strip() for line in f]

class Monkey:
    def __init__(self, mokey_id, items, operation, test, true, false):
        self.items = list(map(int, re.findall('[0-9]+', items)))
        self.operation = '(' + re.search('old(.*)', operation)[0] + ')'
        self.test = int(re.search('[0-9]+', test)[0])
        self.true = int(re.search('[0-9]+', true)[0])
        self.false = int(re.search('[0-9]+', false)[0])
        
    def inspect(self, relief_op):
        items = [eval(self.operation + relief_op) for old in self.items]
        where = [self.true if (item % self.test) == 0 else self.false for item in items]
        self.items = []
        return items, where
    
monkeys = [Monkey(*monkey) for monkey in zip(*(iter(d for d in data if len(d)),) * 6)]

activity = [0 for _ in monkeys]
for i in range(20):
    for j, m in enumerate(monkeys):
        items, where = m.inspect('//3')
        activity[j] += len(items)
        for item, to in zip(items, where):
            monkeys[to].items.append(item)
            
prod(sorted(activity)[-2:])

76728

In [26]:
monkeys = [Monkey(*monkey) for monkey in zip(*(iter(d for d in data if len(d)),) * 6)]

relief = str(prod(m.test for m in monkeys))

activity = [0 for _ in monkeys]
for i in range(10000):
    for j, m in enumerate(monkeys):
        items, where = m.inspect('%' + relief)
        activity[j] += len(items)
        for item, to in zip(items, where):
            monkeys[to].items.append(item)
            
prod(sorted(activity)[-2:])

21553910156